In [169]:
import sympy as sp
import numpy as np
from sympy import diff
from IPython.display import display

In [170]:
from commons import MethodOfOrthonormalFrames,StructureEquations

In [12]:
u,v = sp.symbols("u,v")
du,dv=sp.symbols("du,dv")
duAdv= sp.symbols("duAdv")

In [161]:
class Surface:
    def __init__(self,u,v,p:tuple):
        self.u=u
        self.v=v
        self.p=p
        
        self.p_u=[diff(x,self.u) for x in self.p]
        self.p_v=[diff(x,self.v) for x in self.p]
        
        self.p_uu=[diff(x,self.u) for x in self.p_u]
        self.p_uv=[diff(x,self.v) for x in self.p_u]
        self.p_vv=[diff(x,self.v) for x in self.p_v]
        
        E=np.dot(self.p_u,self.p_u)
        F=np.dot(self.p_u,self.p_v)
        G=np.dot(self.p_v,self.p_v)
        self.E=E
        self.F=F
        self.G=G
        
        e=np.cross(self.p_u,self.p_v)
        e_mod=sp.sqrt(np.dot(e,e))
        self.e=[(x/e_mod).simplify() for x in e]
        
        L=np.dot(self.p_uu,self.e)
        M=np.dot(self.p_uv,self.e)
        N=np.dot(self.p_vv,self.e)
        self.L=L
        self.M=M
        self.N=N
        self.K = (L*N-M**2)/(E*G-F**2)
        self.H = (E*N+G*L-2*F*M)/(2*E*G-2*F**2)
        self.K=self.K.simplify()
        self.H=self.H.simplify()
        k1,k2=sp.symbols("k1,k2")
        
        k1k2=sp.solve([
            k1*k2-self.K,
            (k1+k2)-2*self.H
        ],(k1,k2))
        self.k1=k1k2[0][0]
        self.k2=k1k2[0][1]
        
    def val(self,u,v):
        return tuple(x.subs(self.u,u).subs(self.v,v).evalf() for x in self.p)

    def e_val(self,u,v):
        return tuple(x.subs(self.u,u).subs(self.v,v).evalf() for x in self.e)
    
    def I(self,du,dv):
        return (self.E*du*du + 2*self.F*du*dv + self.G*dv*dv).simplify()
    
    def II(self,du,dv):
        return (self.L*du*du + 2*self.M*du*dv + self.N*dv*dv).simplify()
    
    def III(self,du,dv):
        return (-self.K*self.I(du,dv)+2*self.H*self.II(du,dv)).simplify()
    
    def principal_directions(self,u0,v0):
        k1,k2=self.k1,self.k2
        
        E,F,G,L,M,N=self.E,self.F,self.G,self.L,self.M,self.N
        k1,k2,E,F,G,L,M,N=[m.subs(self.u,u0).subs(self.v,v0) for m in [k1,k2,E,F,G,L,M,N]]
        
        a,b=sp.symbols("a,b")
        #II(w1) = k1; II(w2) = k2
        #E(a,b) da da + 2 F da db + G db db = k1
        #E(a,b) (da/du)**2 * dudu + ... = k1
        
        p_u=sp.Array(self.p_u).subs(self.u,u0).subs(self.v,v0)
        p_v=sp.Array(self.p_v).subs(self.u,u0).subs(self.v,v0)
        w=sp.solve([
            L*a + M*b -k1*(E*a + F*b),
            M*a + N*b -k1*(F*a + G*b),
            np.dot(a*p_u+b*p_v,a*p_u+b*p_v)-1
        ],(a,b))
        
        w1=w[0][0]*p_u+w[0][1]*p_v
        w2=w[1][0]*p_u+w[1][1]*p_v
        return w1,w2
    

class SurfaceCurve:
    def __init__(self, u,v,s, surface, curve:list):
        self.u=u
        self.v=v
        if isinstance(surface,list):
            self.surface=Surface(u,v,surface)
        else:
            assert surface.u==u and surface.v==v
            self.surface=surface
        # u(s), v(s)
        self.curve=curve
    
    def normal_curvature(self):
        pass
    
    def geodesic_curvature(self):
        pass
    
    

In [162]:
surface=Surface(u,v,[
    sp.cos(u)*sp.cos(v),
    sp.cos(u)*sp.sin(v),
    sp.sin(u)
])

In [163]:
w1,w2=surface.principal_directions(0,1)
display(w1)
display(w2)

[sin(1)/sqrt(cos(1)**2 + sin(1)**2), -cos(1)/sqrt(cos(1)**2 + sin(1)**2), 0]

[-sin(1)/sqrt(cos(1)**2 + sin(1)**2), cos(1)/sqrt(cos(1)**2 + sin(1)**2), 0]

In [164]:
s=sp.symbols("s")

c1=SurfaceCurve(u,v,s,[
    u*sp.cos(v),
    u*sp.sin(v),
    v
],[
    sp.sqrt(v),
    -v+sp.cos(v)
])

In [168]:
np.dot(w2,w2).simplify()

1